In [1]:
!pip install pymongo
import pymongo
cli = pymongo.MongoClient(host='34.212.59.71')

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from IPython import display
from scipy.spatial.distance import cosine

In [3]:
wiki_mongo_database = cli.wiki_mongo_database
wiki_mongo_collection = cli.wiki_mongo_database.wiki_mongo_collection
wiki_mongo_collection.count()

5665

In [4]:
list_of_docs=[]
df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))

### NLP - Nearest Neighbor

In [5]:
from spacy.en import English
nlp = English()
content_vec = nlp(df['clean_content'][0]).vector
content_vec.shape

(300,)

In [6]:
content_vecs = np.array([nlp(i).vector for i in df['clean_content']])
content_vecs.shape

(5665, 300)

In [7]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=5)
nn.fit(content_vecs)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [9]:
def most_similar(search):
    distance, indices = nn.kneighbors(nlp(search).vector.reshape(1,-1))
    
    return df.ix[indices[0]][['category','title','clean_content']]

In [10]:
most_similar('map')

,category,title,clean_content
1426,Geographic information systems,Internet Map Server,internet map server ims provide map internet u...
1290,Geographic information systems,Earth Point,earthpoint map southwest idaho real estate lis...
1460,Geographic information systems,Map regression,map regression process work backwards later ma...
1271,Geographic information systems,David Rumsey Historical Map Collection,david rumsey historical map collection world l...
1390,Geographic information systems,Google Maps,google map web mapping service develop google ...


In [8]:
nn.kneighbors(nlp('yoga').vector.reshape(1,-1))

(array([[ 6.0041664 ,  6.00818783,  6.14223797,  6.14609059,  6.18124972]]),
 array([[5657, 3102, 5574, 5579, 5479]]))

In [18]:
import pickle
!pip install redis
import redis
redis_ip = '34.212.59.71'

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
## set the pickled nearest neighbors model

r = redis.StrictRedis(redis_ip)
model = pickle.dumps(nn)
r.set('model_nn', model)

True

In [20]:
## set the pickled original DataFrame with cleaned data

data = pickle.dumps(df)
r.set('tbl_wiki_raw_data', data)

True

In [21]:
#r.flushall()
r.keys()

[b'model_nn',
 b'data',
 b'nlp_cat_vecs',
 b'tbl_vocabulary_expression',
 b'tbl_LSA',
 b'nlp_content_vecs',
 b'tbl_wiki_raw_data']

In [ ]:
pickle.loads(r.get('model_nn'))

In [ ]:
pickle.loads(r.get('tbl_wiki_raw_data')).head()